In [ ]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
!pip install tensorflow-model-optimization
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow.keras.models import load_model
from google.colab import files
import shutil
import gc
from keras.datasets import cifar100
from tensorflow.keras import layers
import shutil
from google.colab import drive
drive.mount('/content/drive')

np.random.seed(42)
tf.random.set_seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 3.8 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
num_classes = 100
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = cifar100.load_data()


x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



169001437/169001437 [==============================] - 3s 0us/step
x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


In [ ]:
model_name = 'resnet'
dataset = 'CIFAR100'

---

# Distillation

In [ ]:
def train_custom(model, train_data, train_labels, test_data, test_labels):
    initailsoftmax = model.student.predict(test_data)
    model.student.save_weights(init_weights)
    model.student.save(init_model)
    initaildf = pd.DataFrame(initailsoftmax)
    filename = filepath + '0_softmax.csv'
    initaildf.to_csv(filename,index=False)
    epoch = 1
    while epoch < 26:
        print(f"Epoch {epoch}:")

        # Training on one epoch
        model.fit(train_data, train_labels, epochs=1, batch_size=64, verbose=1)

        # Evaluate on the test dataset
        results = model.student.predict(test_data)
        softmax_df = pd.DataFrame(results)
        filename = filepath+str(epoch)+'_softmax.csv'
        softmax_df.to_csv(filename,index=False)
        loss,accuracy = model.student.evaluate(test_data,test_labels)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

        if epoch ==25:
          model.student.save_weights(final_weights)
          model.student.save(final_model)
        epoch += 1

# TO DO EVALUATE FUNCTION


In [ ]:
class Distiller(tf.keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha,
        temperature,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )

        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)
        print(y_prediction)
        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results

In [ ]:
teacher_model_path = '/content/base/resnetCIFAR100_final.tf'
techer_model = load_model(teacher_model_path)

# Student 0.1 Alpha

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model_path = '/content/base/resnetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 7s 12ms/step
Epoch 1:
313/313 [==============================] - 5s 11ms/step - loss: 3.8687 - accuracy: 0.1101
Test Loss: 3.8686816692352295, Test Accuracy: 0.11010000109672546
Epoch 2:
313/313 [==============================] - 4s 11ms/step - loss: 3.5398 - accuracy: 0.1629
Test Loss: 3.5398473739624023, Test Accuracy: 0.16290000081062317
Epoch 3:
313/313 [==============================] - 4s 12ms/step - loss: 3.5184 - accuracy: 0.1689
Test Loss: 3.518399238586426, Test Accuracy: 0.1688999980688095
Epoch 4:
313/313 [==============================] - 4s 12ms/step - loss: 3.6055 - accuracy: 0.1787
Test Loss: 3.605503559112549, Test Accuracy: 0.17870000004768372
Epoch 5:
313/313 [==============================] - 4s 11ms/step - loss: 3.3427 - accuracy: 0.2116
Test Loss: 3.3427469730377197, Test Accuracy: 0.21160000562667847
Epoch 6:
313/313 [==============================] - 4s 11ms/step - loss: 3.1370 - accuracy: 0.2472
Test Loss: 3.1369850635

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same", 'zip', "student_same")
colab_link = "/content/student_same.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C100/student_same.zip'

# Student 0.1 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)

model_path = '/content/base/resnetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 6s 14ms/step
Epoch 1:
313/313 [==============================] - 4s 11ms/step - loss: 3.8570 - accuracy: 0.1100
Test Loss: 3.857017993927002, Test Accuracy: 0.10999999940395355
Epoch 2:
313/313 [==============================] - 3s 11ms/step - loss: 3.6088 - accuracy: 0.1551
Test Loss: 3.608776569366455, Test Accuracy: 0.1551000028848648
Epoch 3:
313/313 [==============================] - 3s 11ms/step - loss: 3.7386 - accuracy: 0.1668
Test Loss: 3.7385640144348145, Test Accuracy: 0.16680000722408295
Epoch 4:
313/313 [==============================] - 3s 11ms/step - loss: 3.3875 - accuracy: 0.1902
Test Loss: 3.3874855041503906, Test Accuracy: 0.19020000100135803
Epoch 5:
313/313 [==============================] - 4s 12ms/step - loss: 3.2083 - accuracy: 0.2189
Test Loss: 3.208308696746826, Test Accuracy: 0.21889999508857727
Epoch 6:
313/313 [==============================] - 4s 11ms/step - loss: 2.9293 - accuracy: 0.2771
Test Loss: 2.92932868003

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_2", 'zip', "student_same_2")
colab_link = "/content/student_same_2.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C100/student_same_2.zip'

# Student 0.1 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)

model_path = '/content/base/resnetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=3,
)
filepath = '/content/student_same_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 4s 9ms/step
Epoch 1:
313/313 [==============================] - 4s 11ms/step - loss: 3.8997 - accuracy: 0.1042
Test Loss: 3.8996691703796387, Test Accuracy: 0.10419999808073044
Epoch 2:
313/313 [==============================] - 4s 12ms/step - loss: 3.5795 - accuracy: 0.1574
Test Loss: 3.579460859298706, Test Accuracy: 0.1573999971151352
Epoch 3:
313/313 [==============================] - 4s 13ms/step - loss: 3.6847 - accuracy: 0.1646
Test Loss: 3.6846671104431152, Test Accuracy: 0.16459999978542328
Epoch 4:
313/313 [==============================] - 4s 14ms/step - loss: 3.2748 - accuracy: 0.2206
Test Loss: 3.274827003479004, Test Accuracy: 0.22059999406337738
Epoch 5:
313/313 [==============================] - 4s 13ms/step - loss: 3.8131 - accuracy: 0.1747
Test Loss: 3.8131000995635986, Test Accuracy: 0.17470000684261322
Epoch 6:
313/313 [==============================] - 4s 13ms/step - loss: 3.1873 - accuracy: 0.2640
Test Loss: 3.18734669685

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_3", 'zip', "student_same_3")
colab_link = "/content/student_same_3.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C100/student_same_3.zip'

# Student same Alpha 0.5

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model_path = '/content/base/resnetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 6s 11ms/step
Epoch 1:
313/313 [==============================] - 5s 11ms/step - loss: 3.8146 - accuracy: 0.1113
Test Loss: 3.8145995140075684, Test Accuracy: 0.11129999905824661
Epoch 2:
313/313 [==============================] - 3s 11ms/step - loss: 3.6379 - accuracy: 0.1528
Test Loss: 3.6378531455993652, Test Accuracy: 0.15279999375343323
Epoch 3:
313/313 [==============================] - 4s 11ms/step - loss: 3.6333 - accuracy: 0.1747
Test Loss: 3.6333210468292236, Test Accuracy: 0.17470000684261322
Epoch 4:
313/313 [==============================] - 4s 13ms/step - loss: 3.4028 - accuracy: 0.2059
Test Loss: 3.4027585983276367, Test Accuracy: 0.20589999854564667
Epoch 5:
313/313 [==============================] - 4s 13ms/step - loss: 3.5376 - accuracy: 0.2180
Test Loss: 3.537592887878418, Test Accuracy: 0.21799999475479126
Epoch 6:
313/313 [==============================] - 4s 12ms/step - loss: 3.1216 - accuracy: 0.2541
Test Loss: 3.12158846

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50", 'zip', "student_same_50")
colab_link = "/content/student_same_50.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C100/student_same_50.zip'

# Student 0.5 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)

model_path = '/content/base/resnetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 5s 10ms/step
Epoch 1:
313/313 [==============================] - 6s 13ms/step - loss: 3.8538 - accuracy: 0.1054
Test Loss: 3.853776454925537, Test Accuracy: 0.10540000349283218
Epoch 2:
313/313 [==============================] - 4s 12ms/step - loss: 3.5966 - accuracy: 0.1504
Test Loss: 3.596560478210449, Test Accuracy: 0.15039999783039093
Epoch 3:
313/313 [==============================] - 4s 12ms/step - loss: 3.4087 - accuracy: 0.1886
Test Loss: 3.4086661338806152, Test Accuracy: 0.18860000371932983
Epoch 4:
313/313 [==============================] - 4s 14ms/step - loss: 3.4122 - accuracy: 0.2008
Test Loss: 3.4121735095977783, Test Accuracy: 0.20080000162124634
Epoch 5:
313/313 [==============================] - 4s 13ms/step - loss: 3.1398 - accuracy: 0.2443
Test Loss: 3.139788866043091, Test Accuracy: 0.2442999929189682
Epoch 6:
313/313 [==============================] - 4s 14ms/step - loss: 3.1609 - accuracy: 0.2501
Test Loss: 3.16086339950

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_2", 'zip', "student_same_50_2")
colab_link = "/content/student_same_50_2.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C100/student_same_50_2.zip'

# Student 0.5 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)

model_path = '/content/base/resnetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)
distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.5,
    temperature=3,
)
filepath = '/content/student_same_50_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'
!mkdir '/content/student_same_50_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 4s 10ms/step
Epoch 1:
313/313 [==============================] - 6s 14ms/step - loss: 3.8749 - accuracy: 0.1054
Test Loss: 3.8748891353607178, Test Accuracy: 0.10540000349283218
Epoch 2:
313/313 [==============================] - 5s 14ms/step - loss: 3.5849 - accuracy: 0.1548
Test Loss: 3.5849075317382812, Test Accuracy: 0.15479999780654907
Epoch 3:
313/313 [==============================] - 4s 12ms/step - loss: 3.6732 - accuracy: 0.1559
Test Loss: 3.673161745071411, Test Accuracy: 0.1559000015258789
Epoch 4:
313/313 [==============================] - 4s 12ms/step - loss: 3.4800 - accuracy: 0.1848
Test Loss: 3.4799981117248535, Test Accuracy: 0.18479999899864197
Epoch 5:
313/313 [==============================] - 4s 12ms/step - loss: 3.1955 - accuracy: 0.2321
Test Loss: 3.195542335510254, Test Accuracy: 0.2320999950170517
Epoch 6:
313/313 [==============================] - 4s 11ms/step - loss: 3.0889 - accuracy: 0.2560
Test Loss: 3.08894085884

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_50_3", 'zip', "student_same_50_3")
colab_link = "/content/student_same_50_3.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C100/student_same_50_3.zip'

# Student same Alpha 0.9

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)
model_path = '/content/base/resnetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 5s 9ms/step
Epoch 1:
313/313 [==============================] - 5s 12ms/step - loss: 3.9170 - accuracy: 0.0997
Test Loss: 3.9169538021087646, Test Accuracy: 0.09969999641180038
Epoch 2:
313/313 [==============================] - 4s 11ms/step - loss: 3.7407 - accuracy: 0.1466
Test Loss: 3.7406859397888184, Test Accuracy: 0.14659999310970306
Epoch 3:
313/313 [==============================] - 4s 13ms/step - loss: 3.6120 - accuracy: 0.1639
Test Loss: 3.6119983196258545, Test Accuracy: 0.1639000028371811
Epoch 4:
313/313 [==============================] - 4s 11ms/step - loss: 3.2928 - accuracy: 0.2183
Test Loss: 3.29276442527771, Test Accuracy: 0.218299999833107
Epoch 5:
313/313 [==============================] - 4s 13ms/step - loss: 3.3802 - accuracy: 0.2279
Test Loss: 3.38023042678833, Test Accuracy: 0.22789999842643738
Epoch 6:
313/313 [==============================] - 4s 13ms/step - loss: 3.3435 - accuracy: 0.2159
Test Loss: 3.343490600585937

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90", 'zip', "student_same_90")
colab_link = "/content/student_same_90.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C100/student_same_90.zip'

# Student 0.9 (2)

In [ ]:
np.random.seed(24)
tf.random.set_seed(24)
model_path = '/content/base/resnetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_2/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_2/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 7s 10ms/step
Epoch 1:
313/313 [==============================] - 6s 14ms/step - loss: 3.8449 - accuracy: 0.1052
Test Loss: 3.844937562942505, Test Accuracy: 0.10520000010728836
Epoch 2:
313/313 [==============================] - 4s 13ms/step - loss: 3.5983 - accuracy: 0.1613
Test Loss: 3.5982792377471924, Test Accuracy: 0.16130000352859497
Epoch 3:
313/313 [==============================] - 4s 12ms/step - loss: 3.4039 - accuracy: 0.1848
Test Loss: 3.403855562210083, Test Accuracy: 0.18479999899864197
Epoch 4:
313/313 [==============================] - 4s 13ms/step - loss: 3.5605 - accuracy: 0.1872
Test Loss: 3.5605406761169434, Test Accuracy: 0.18719999492168427
Epoch 5:
313/313 [==============================] - 4s 14ms/step - loss: 3.5931 - accuracy: 0.1839
Test Loss: 3.5930843353271484, Test Accuracy: 0.18389999866485596
Epoch 6:
313/313 [==============================] - 4s 12ms/step - loss: 2.7915 - accuracy: 0.3093
Test Loss: 2.791524887

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_2", 'zip', "student_same_90_2")
colab_link = "/content/student_same_90_2.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C100/student_same_90_2.zip'

# Student 0.9 (3)

In [ ]:
np.random.seed(2)
tf.random.set_seed(2)
model_path = '/content/base/resnetCIFAR100_init.tf'
student = load_model(model_path)
student_scratch = tf.keras.models.clone_model(student)

distiller = Distiller(student=student, teacher=techer_model)
distiller.compile(
    optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
    metrics=[tf.keras.metrics.CategoricalCrossentropy()],
    student_loss_fn=tf.keras.losses.CategoricalCrossentropy(),
    distillation_loss_fn=tf.keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=3,
)

filepath = '/content/student_same_90_3/'
init_weights = filepath + model_name + dataset + '_init_weights.tf'
init_model = filepath + model_name + dataset + '_init.tf'
final_weights = filepath + model_name + dataset + '_final_weights.tf'
final_model= filepath + model_name + dataset + 'final.tf'

!mkdir '/content/student_same_90_3/'
train_custom(distiller, x_train,y_train,x_test,y_test)

313/313 [==============================] - 4s 10ms/step
Epoch 1:
313/313 [==============================] - 5s 14ms/step - loss: 3.8213 - accuracy: 0.1190
Test Loss: 3.8212859630584717, Test Accuracy: 0.11900000274181366
Epoch 2:
313/313 [==============================] - 4s 11ms/step - loss: 3.6414 - accuracy: 0.1491
Test Loss: 3.641357421875, Test Accuracy: 0.14910000562667847
Epoch 3:
313/313 [==============================] - 4s 13ms/step - loss: 3.5372 - accuracy: 0.1732
Test Loss: 3.537216901779175, Test Accuracy: 0.17319999635219574
Epoch 4:
313/313 [==============================] - 4s 12ms/step - loss: 3.2853 - accuracy: 0.2172
Test Loss: 3.2852866649627686, Test Accuracy: 0.21719999611377716
Epoch 5:
313/313 [==============================] - 4s 11ms/step - loss: 3.3436 - accuracy: 0.2199
Test Loss: 3.343604326248169, Test Accuracy: 0.2198999971151352
Epoch 6:
313/313 [==============================] - 4s 11ms/step - loss: 3.3983 - accuracy: 0.2240
Test Loss: 3.39826250076293

In [ ]:
del (student)
gc.collect()
keras.backend.clear_session()
shutil.make_archive("student_same_90_3", 'zip', "student_same_90_3")
colab_link = "/content/student_same_90_3.zip"
gdrive_link = "/content/drive/MyDrive/ResNet_D_C100"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/ResNet_D_C100/student_same_90_3.zip'

---